# Benchmarking circuit evaluation
Amazon Braket SV1 vs Pennylane-Lightning-GPU

In [1]:
!qbraid jobs enable pennylane_gpu

Successfully enabled qBraid Quantum Jobs in the pennylane_gpu environment.
Every AWS job you run will now be submitted through the qBraid API, so no access keys/tokens are necessary. 

To disable, run: `qbraid jobs disable pennylane_gpu`


In [2]:
!qbraid envs list

# installed environments:
#
default                  jobs  /opt/.qbraid/environments/qbraid_000000
qsharp                         /opt/.qbraid/environments/qsharp_b54crn
intel                          /opt/.qbraid/environments/intel_zr7hfq
bloqade                        /opt/.qbraid/environments/bloqade_374m88
qiskit_gpu                     /home/jovyan/.qbraid/environments/qiskit_gpu_tyt64d
tensorflow                     /home/jovyan/.qbraid/environments/tensorflow_uorhf3
pennylane_gpu            jobs  /home/jovyan/.qbraid/environments/pennylane_gpu_21uik3
qbraid_sdk               jobs  /home/jovyan/.qbraid/environments/qbraid_sdk_9j9sjy



In [3]:
!nvidia-smi

Thu Jul 20 01:30:43 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.182.03   Driver Version: 470.182.03   CUDA Version: 11.5     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            On   | 00000000:00:1E.0 Off |                    0 |
| N/A   36C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
import pennylane as qml
from pennylane import numpy as np

qml.about()

Name: PennyLane
Version: 0.31.0
Summary: PennyLane is a Python quantum machine learning library by Xanadu Inc.
Home-page: https://github.com/PennyLaneAI/pennylane
Author: 
Author-email: 
License: Apache License 2.0
Location: /home/jovyan/.qbraid/environments/pennylane_gpu_21uik3/pyenv/lib/python3.9/site-packages
Requires: appdirs, autograd, autoray, cachetools, networkx, numpy, pennylane-lightning, requests, rustworkx, scipy, semantic-version, toml
Required-by: amazon-braket-pennylane-plugin, PennyLane-Lightning, PennyLane-Lightning-GPU

Platform info:           Linux-5.4.247-162.350.amzn2.x86_64-x86_64-with-glibc2.31
Python version:          3.9.12
Numpy version:           1.23.5
Scipy version:           1.10.0
Installed devices:
- default.gaussian (PennyLane-0.31.0)
- default.mixed (PennyLane-0.31.0)
- default.qubit (PennyLane-0.31.0)
- default.qubit.autograd (PennyLane-0.31.0)
- default.qubit.jax (PennyLane-0.31.0)
- default.qubit.tf (PennyLane-0.31.0)
- default.qubit.torch (PennyLa

In [5]:
n_wires = 12

dev_braket = dev = qml.device(
    "braket.aws.qubit",
    device_arn="arn:aws:braket:::device/quantum-simulator/amazon/sv1",
    wires=n_wires,
)
dev_gpu = qml.device("lightning.gpu", wires=n_wires)

In [6]:
def circuit(params):
    for i in range(n_wires):
        qml.RX(params[i], wires=i)
    for i in range(n_wires):
        qml.CNOT(wires=[i, (i + 1) % n_wires])

    # Measure all qubits
    observables = [qml.PauliZ(n_wires - 1)] + [
        qml.Identity(i) for i in range(n_wires - 1)
    ]
    return qml.expval(qml.operation.Tensor(*observables))

In [7]:
qnode_braket = qml.QNode(circuit, dev_braket)
qnode_gpu = qml.QNode(circuit, dev_gpu)

In [8]:
import time

params = np.random.random(n_wires)

t_0_braket = time.time()
qnode_braket(params)
t_1_braket = time.time()

t_0_gpu = time.time()
qnode_gpu(params)
t_1_gpu = time.time()

print("Execution time on Braket SV1 device (seconds):", t_1_braket - t_0_braket)

print("Execution time on Lightning GPU device (seconds):", t_1_gpu - t_0_gpu)

Execution time on Braket SV1 device (seconds): 12.66253662109375
Execution time on Lightning GPU device (seconds): 0.9639229774475098
